In [1]:
import cv2
import pandas as pd
import numpy as np

In [2]:
camera = cv2.VideoCapture(1)

if not camera.isOpened():
    camera = cv2.VideoCapture(0)
if not camera.isOpened():
    raise IOError("Error 404: Can't find Camera")

# cv2.namedWindow("Image Capturer", cv2.WINDOW_NORMAL)
while True:
    ret,frame = camera.read()
    if not ret:
        print("Capture Failed")
        break
    cv2.imshow("Image Capture",frame)
    
    key = cv2.waitKey(1)
    if key%256 == 27: #Esc to exit
        print("Exiting...")
        break
    elif key%256 == 32: #Spacebar to capture
        imageID = "Collected Image.jpg"
        cv2.imwrite(imageID, frame)
        print("{} written!".format(imageID))
        break
        
camera.release()
cv2.destroyAllWindows()

Collected Image.jpg written!


In [3]:
image = cv2.imread("Collected Image.jpg")
pd.read_csv("colors.csv").head()

,air_force_blue_raf,Air Force Blue (Raf),#5d8aa8,93,138,168
0,air_force_blue_usaf,Air Force Blue (Usaf),#00308f,0,48,143
1,air_superiority_blue,Air Superiority Blue,#72a0c1,114,160,193
2,alabama_crimson,Alabama Crimson,#a32638,163,38,56
3,alice_blue,Alice Blue,#f0f8ff,240,248,255
4,alizarin_crimson,Alizarin Crimson,#e32636,227,38,54


In [4]:
onClick = False
r = g = b = xpos = ypos = 0
index = ["color", "color_name", "hex", "R", "G", "B"]
Colorcsv = pd.read_csv("colors.csv",names=index, header= None)

In [5]:
def getColorName(R,G,B):
    minimum = 10000
    for i in range (len(Colorcsv)):
        data = abs(R - int(Colorcsv.loc[i, "R"])) + abs(G - int(Colorcsv.loc[i, "G"])) + abs(B - int(Colorcsv.loc[i, "B"]))
        if data <= minimum: 
            minimum = data
            cname = Colorcsv.loc[i,"color_name"]
    return cname

In [6]:
def leftOnClick (event, x, y, flags, param):
    if event == cv2.EVENT_LBUTTONDBLCLK:
        global b, g, r, xpos, ypos, onClick
        onClick = True
        xpos = x 
        ypos = y
        b,g,r = image[y,x]
        b = int(b)
        g = int(g)
        r = int(r)

In [7]:
cv2.namedWindow('image')
cv2.setMouseCallback('image', leftOnClick)

while True:
    cv2.imshow("image", image)
    if onClick:
        cv2.rectangle(image,(20, 20), (750, 60), (b, g, r), -1)
        text = getColorName(r, g, b) + ' R=' + str(r) + ' G=' + str(g) + ' B=' + str(b)
        cv2.putText(image, text, (50, 50), 2, 0.8, (255, 255, 255), 2, cv2.LINE_AA)
        if r + g + b >= 600:
            cv2.putText(image, text, (50, 50), 2, 0.8, (0, 0, 0), 2, cv2.LINE_AA)
        onClick = False
        
    if cv2.waitKey(20) & 0xFF == 27:
        break

cv2.destroyAllWindows()